In [ ]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

import PyPDF2
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

In [ ]:
load_dotenv(override = True)
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    print("No API key")
elif not api_key.startswith("sk-proj-"):
    print("API key doesn't look correct, check it")
elif api_key.strip() != api_key:
    print("It looks like API key has an extra space - check it")
else:
    print("API key looks good, moving on!")

In [ ]:
openai = OpenAI()

In [ ]:
# path to the CV
path = "/Users/yanasklar/Documents/For applying/CV/СV_YanaSklyar_c.pdf"
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36"
}

class Vacancy:
    def __init__(self, url, instructions = ""):
        self.url = url
        
        # configure Chrome settings
        options = Options()
        # options.add_argument("--headless")  
        """
        Headless mode runs the browser in the background (invisible).
        However, some websites (like openai.com) block headless browsers.
        So if this line is active, the page may not load correctly and you may not get the full content.
        """
        options.add_argument("--disable-gpu")
        options.add_argument("--no-sandbox")
        options.add_argument("--window-size=1920x1080")

        # use webdriver-manager to manage ChromeDriver
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)
        time.sleep(3) # let the page load

        # take the source of the page
        page_source = driver.page_source
        driver.quit()

        # analyse with BeautifulSoup
        soup = BeautifulSoup(page_source, 'html.parser')

        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["img", "script", "style", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator='\n', strip=True)

        # read CV
        with open(path, 'rb') as f:
            reader = PyPDF2.PdfReader(f)
            cv_text = ""
            for page in reader.pages:
                text = page.extract_text()
                if text:
                    cv_text += text + "\n"
        self.cv_text = cv_text

        # summarise and print the description of the job
        message = f"""Here is the content of a webpage: {self.text}.
        Find job description on that page,
        summarise it, include the list requirements and other important details.
        """
        messages = [{"role":"user", "content":message}]
        response = openai.chat.completions.create(model='gpt-4o-mini', messages = messages)
        print("The job description: ", response.choices[0].message.content)

        # create prompts
        self.system_prompt = """You are a career assistant specializing in writing cover letter.
        Your tasks:
        1. Read the candidate's CV (provided as text).
        2. Read the job description (provided from a webpage).
        3. Write a concise and compelling cover letter, that:
        - Hightlights the most relevant experience and  skills from the CV,
        - Aligns directly wit the requirements in the job description,
        - Adapts to cultural and professional norms in Israel.
        The letter should be no longer than half a page, persuasive and tailored to make the applicant stand out.
        """

        user_prompt = f"""
            Here is my CV:
            {self.cv_text}
            
            The job vacancy is from the website {self.title}.
            Here is the decription of the vacancy:
            {self.text}
            Please write a cover letter that connects my background to this vacancy.
            Make it persuasive and suitable for Israeli job market.
            """
            
        if instructions:
            user_prompt += f"Additional instructions: {instructions}"
        self.user_prompt = user_prompt

In [ ]:
def cover_letter(url, instructions = ""):
    vacancy = Vacancy(url, instructions)
    messages = [
        {"role":"system", "content":vacancy.system_prompt},
        {"role":"user", "content":vacancy.user_prompt}
    ]
    response = openai.chat.completions.create(model='gpt-4o-mini', messages=messages)
    if not response:
        print("smt went wrong")
    print(response.choices[0].message.content)

In [ ]:
a = "https://www.linkedin.com/jobs/view/4285898438/?alternateChannel=search&eBP=CwEAAAGY3R5LOabDLOVTy6xvBcSlWyAkIXQz8IRkSM3rgsqTPtvcEvUSnq980O7oLV2Hh_ldTpc2cBBmRq1IRnLtp7TzEcUvndFEXeCuviA5yo7oFYfW7KoEp4SPNzmf3D9LtnSgk9Iudy3skk6n3hVOtyDpx8Zm0AiTWPvdwCaZ_w5Xu8lAG797NRNDco71ynm99LmCOC9Go7DdDQ2eLewamc4SOsA4xWcXy0GmZVy3kBF1AprK3ylAYR2wrm5-hp4lRpbbfUxXjkEOG6H_GbPpKtN-N8mYnMd9w_cej5qQmTFX86gqSi6HuXFtK0h46TbOS5r-YQksVd1Yb4kYZnDznWXPLbxp04xVJSPzsHoa05wQdOfZ2UUSoMTJmic3n3qfV2u9Bp8n4sLYtINpzKdvm4eADGGkN-nR3O2oPeas9XjGbBwNdjXHAcX_PJoRwlFdQ1gVkYQEF1T7qAfXUJoUt-fv4oLxGnIgV6yJuMgw&refId=9NA7Bvt%2FhCqDkFNRGu1dPA%3D%3D&trackingId=W11hvpcIjHA%2FjU%2FFZ%2B1uAA%3D%3D"
b = "The style of the cover letter should informal, as if i talked to a friend about my background"
cover_letter(a, b)